In [4]:
#cross-correlazione tra finestra con zero-padding e segnale 
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.signal as signal
from numpy.fft import rfft, irfft

arrwnd=np.loadtxt("FinestreRef.npy")

def calcoloCCR(data, arrwnd, plotGrafici = True):

#le due liste da confrontare devono avere stessa lunghezza

    for i in range (len(arrwnd)):
   
        template=arrwnd[i].tolist()
    
    #Zero padding
        for i in range (10,len(data)):
            template.append(0)

    # Fast Fourier Transform (FFT)
        Data= rfft(data)
   
    # Inverto il vettore template
        tempY = []
        while len(template) > 0:
            tempY.append(template.pop())
    
        Y = rfft(tempY)

    # Faccio la formula inversa della FFT
        colpoDiSonno = False
        CrossCorrelazione = irfft(Data*Y)
        if(max(CrossCorrelazione)>4):
            colpoDiSonno = True
            print("Avvenuto colpo di sonno")
            if plotGrafici:
                x=np.arange(0,len(data)-1)
                plt.plot(x,CrossCorrelazione)
                plt.xlabel("Campioni")
                plt.ylabel("CrossCorrelazione")
                line1=plt.plot(x,[4]*(len(data)-1),'r')
                plt.show()
        return colpoDiSonno

In [11]:
%matplotlib inline
from pynq.pmods import Grove_IMU
from IPython import display
from ipywidgets import widgets
import numpy as np
import scipy.signal as signal
import time
import matplotlib.pyplot as plt

def pitchcalc(accX, accY, accZ):
    argomento=accY/np.sqrt(accX*accX*accZ*accZ)
    pitch=np.arctan(argomento)
    return pitch


# initialize Grove IMU on PMOD 1 (JB) and grove id 4 (GR4).
# PMOD mapping is: PMOD 1 -> JB, PMOD 2 -> JC, PMOD 3 -> JD, PMOD 4 -> JE
# NOTE: only GR4 can be used for the Grove IMU
imu = Grove_IMU(1,4)

#empty lists creation
accX=list()
accY=list()
accZ=list()
pitch_l=list()

i=0

while (True):
    try:
        
        motion = imu.get_motion()
        
        start=time.time()
        
        # get data from accelerometer
        accX.append(motion[0])
        accY.append(motion[1])
        accZ.append(motion[2])
        pitch_l.append(pitchcalc(motion[0],motion[1],motion[2]))
        
        time.sleep(0.03)
        i=i+1
            
        end=time.time()
        
        while(i<100):
            datiZ=accZ[(len(accZ)-1-i):len(accZ)]
            colpodiSonno=calcoloCCR(datiZ,arrwnd,False)
            
        else:
            datiZ=accZ[(len(accZ)-101):len(accZ)]
            colpodiSonno=calcoloCCR(datiZ,arrwnd,False)
    
        print(colpodiSonno)  
    
    except KeyboardInterrupt:
    # exit without showing errors if the stop button in Jupyter is pressed
        break


In [12]:
print(accZ)


[0.89]
